In [ ]:
import numpy as np, scipy.optimize
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

In [ ]:
# Set up time array 
Nd   = 300                  # Number of days in timeseries
dt   = 1/24                 # Time steps in days
time = np.arange(0,Nd,dt)   # Set up time array

In [ ]:
# Set up two-component signal attributes
A1 = 0.08;      A2 = 0.03     # Fractional variation amplitude
f1 = 0.0791;    f2 = 0.1535   # Variation frequencies (cycles / day)
p1 = -0.008;    p2 = -0.54    # Phase at t = 0

In [ ]:
# Create two-component signal
sig = A1*np.sin(2*np.pi*f1*time+p1)       # First signal component
sig = sig + A2*np.sin(2*np.pi*f2*time+p2) # Add in second signal component

In [ ]:
# Plot Signal
fig, ax = plt.subplots()
ax.plot(time,sig)
ax.set_xlabel('Time (days)', fontsize=16)
ax.set_ylabel('Fractional Flux Deviation (mmag)', fontsize=16)
for label in (ax.get_xticklabels() + ax.get_yticklabels()): label.set_fontsize(14)
ax.set_xlim(-5,305)
ax.set_ylim(-0.12,0.12)
ax.invert_yaxis()

In [ ]:
# Define partial-section of signal to autocorrelate across full signal
sld    = 50              # Section length in day
seclen = sld*int(1/dt)   # Number of points in this section
sec    = sig[0:seclen]   # Extract this section

In [ ]:
# Initialise autocorrelation (AC) array list
acorr  = []

In [ ]:
for i in range(0,int(((Nd-sld)/dt)-1)):
    
    sigsec = sig[i:i+seclen]
    var1   = sec-np.mean(sec)
    var2   = sigsec-np.mean(sigsec)
    Pcoeff = np.sum((var1/np.std(sec))*var2/np.std(sigsec))/(len(sec)-1)
    
    acorr.append(Pcoeff)

In [ ]:
# AC time-lag array
tcorr = np.arange(0,(len(acorr))*dt,dt)

In [ ]:
# Find indices of peaks above 0.95
peaks, _ = find_peaks(acorr, height=0.95)

In [ ]:
# Find AC values and corresponding time stamps for those peaks
acp = [acorr[i] for i in peaks]
tcp = [tcorr[i] for i in peaks]

In [ ]:
# Repetition time is the timestamp at the highest AC value of those peaks
reptime = tcp[acp.index(max(acp))]

In [ ]:
print('Repetition time: {0:.4f} days.'.format(reptime))

In [ ]:
fig = plt.figure(figsize = (8,8))  # Figure handle

# Signal Function Plot
ax  = fig.add_subplot(2,1,1)       # Set up axes handle
ax.plot(time,sig)
ax.set_xlabel('Time (days)', fontsize=16)
ax.set_ylabel('Fractional Flux Deviation (mmag)', fontsize=16)
for label in (ax.get_xticklabels() + ax.get_yticklabels()): label.set_fontsize(14)
ax.set_xlim(-5,305)
ax.set_ylim(-0.12,0.12)
ax.plot([0,0],[-1.5,1.5],':',color='black')
ax.plot([reptime,reptime],[-1.5,1.5],':',color='black')
plt.text(reptime+2, -0.09, "Repetition time:\n{0:.2f} days.".format(reptime))
ax.invert_yaxis()

# Autocorrelation Function Plot
ax  = fig.add_subplot(2,1,2)       # Set up axes handle
ax.plot(tcorr,acorr)
ax.set_xlabel('Lag time (days)', fontsize=16)
ax.set_ylabel('Pearson Coefficient', fontsize=16)
for label in (ax.get_xticklabels() + ax.get_yticklabels()): label.set_fontsize(14)
ax.set_xlim(-5,305)
ax.set_ylim(-1.2,1.2)
ax.plot([0,0],[-1.5,1.5],':',color='black')
ax.plot([reptime,reptime],[-1.5,1.5],':',color='black')

In [ ]:
# Section of signal that is correlated across full signal
fig, ax = plt.subplots()
ax.plot(time[0:seclen],sig[0:seclen])
ax.set_xlabel('Time (days)', fontsize=16)
ax.set_ylabel('Fractional Flux Deviation (mmag)', fontsize=16)
for label in (ax.get_xticklabels() + ax.get_yticklabels()): label.set_fontsize(14)
ax.invert_yaxis()

In [ ]:
############################
# Alternative, direct method

sfrac = 1/6         # Use first 1/6 of signal as partial section to autocorrelate
cfrac = 1 - sfrac   # Fraction of signal up to which to calculate AC lag time

acorr = np.array([1]+[np.corrcoef(sig[:-i], sig[i:])[0,1]  \
        for i in range(1, int(np.floor(len(sig)*cfrac)))])

# Autocorrelation Function Plot
fig, ax = plt.subplots()
ax.plot(time[0:int(np.floor(len(sig)*cfrac))]-time[0],acorr)
ax.set_xlim(-5,305)
ax.set_ylim(-1.2,1.2)
ax.plot([0,0],[-1.5,1.5],':',color='black')
ax.plot([reptime,reptime],[-1.5,1.5],':',color='black')
ax.set_xlabel('Lag time (days)', fontsize=16)
ax.set_ylabel('Pearson Coefficient', fontsize=16)